In [1]:
import fiftyone as fo
from fiftyone import ViewField as F
from client.yolov7_client.yolov7_triton_client import YoloV7_Triton_Inference_Client
from client.detectron2_client.detectron2_triton_client import Detectron2_Triton_Client

In [2]:
# start the triton inference server, either locally or on a server

# docker run --gpus all --rm --ipc=host --shm-size=1g --ulimit memlock=-1 --ulimit stack=67108864 \
# -p8000:8000 -p8001:8001 -p8002:8002 --mount type=bind,source="path/to/triton/models",destination=/models \
# nvcr.io/nvidia/tritonserver:22.06-py3 tritonserver --model-repository=/models --strict-model-config=false \
# --log-verbose 1

In [2]:
# make sure in baseline_system directory

%pwd

'c:\\Users\\Alex Lin\\Desktop\\baseline_system'

In [3]:
# initialize the client
yolov7_client = YoloV7_Triton_Inference_Client()
detectron2_client = Detectron2_Triton_Client(width=1344, height=1344)

In [2]:
# start fiftyone

session = fo.launch_app(auto=False)
session.open_tab()


Could not connect session, trying again in 10 seconds

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

In [ ]:
# run the client 

# yolov7_client.infer_dummy()
# yolov7_client.infer_image(input_='data/raw/images', fo_dataset="test_run1")
yolov7_client.infer_video(input_='data/raw/video', fo_dataset="test_run2")

# detectron2_client.infer_dummy()
# detectron2_client.infer_image(input_='data/raw/images/test')
# detectron2_client.infer_video(input_='data/raw/video', output_='data/inference/video')

In [3]:
# load dataset 
dataset_name = "test_run1"
dataset = fo.load_dataset(dataset_name)

In [4]:
# create specific view for low confidence model predictions
low_conf_view = (
    dataset
    .filter_labels("model_detections", F("confidence") < 0.6)
    .sort_by(F("model_detections.confidence"), reverse=True)
)

sample_id = low_conf_view.head(3)
view = dataset.select(sample_id)

In [8]:
# send samples to CVAT

anno_key = "test_run_images"
view.annotate(
    anno_key,
    label_field="ground_truth",
    label_type='detections',
    classes=["person","bicycle","car","motorcycle","airplane","bus","train","truck","boat","traffic light","fire hydrant",
          "stop sign","parking meter","bench","bird","cat","dog","horse","sheep","cow","elephant","bear","zebra",
          "giraffe","backpack","umbrella","handbag","tie","suitcase","frisbee","skis","snowboard","sports ball","kite",
          "baseball bat","baseball glove","skateboard","surfboard","tennis racket","bottle","wine glass","cup","fork",
          "knife","spoon","bowl","banana","apple","sandwich","orange","broccoli","carrot","hot dog","pizza","donut",
          "cake","chair","couch","potted plant","bed","dining table","toilet","tv","laptop","mouse","remote","keyboard",
          "cell phone","microwave","oven","toaster","sink","refrigerator","book","clock","vase","scissors","teddy bear",
          "hair drier", "toothbrush"],
    launch_editor=True,
    url="http://localhost:8080",
    username="django",
    password="bfc",
)

Computing metadata...
 100% |█████████████████████| 2/2 [8.0s elapsed, 0s remaining, 0.2 samples/s]   
Uploading samples to CVAT...
Launching editor at 'http://localhost:8080/tasks/8/jobs/8'...


In [9]:
# merge annotations back to Fiftyone dataset

dataset.load_annotations(anno_key)
view = dataset.load_annotation_view(anno_key)

Download complete
Loading labels for field 'ground_truth'...
 100% |█████████████████████| 2/2 [10.7ms elapsed, 0s remaining, 186.7 samples/s] 

Could not connect session, trying again in 10 seconds



RuntimeError: Client is not connected

In [10]:
# CVAT cleanup

# Delete tasks from CVAT
results = dataset.load_annotation_results(anno_key)
results.cleanup()

# Delete run record (not the labels) from FiftyOne
dataset.delete_annotation_run(anno_key)

Deleting tasks...
 100% |█████████████████████| 1/1 [424.8ms elapsed, 0s remaining, 2.4 samples/s] 


In [ ]:
# delete selected samples
view = dataset.select(session.selected)
dataset.delete_samples(view)